In [1]:
import os
import sys
import math
import torch
import random
import argparse
import traceback
import bittensor as bt
from datasets import load_dataset
from transformers import GPT2TokenizerFast, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, IterableDataset


[WARNING]: failed to patch stdout/stderr for fork-safety: 'OutStream' object
has no attribute 'buffer'
[WARNING]: failed to reconfigure stdout/stderr with custom encoding error
handler: 'OutStream' object has no attribute 'reconfigure'
/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Dataset(IterableDataset):
    def __init__(self, dataset_name:str, tokenizer, sequence_length ):
        self.dataset_name = dataset_name
        self.tokenizer = tokenizer
        self.sequence_length = sequence_length
    def __iter__(self):
        print ('iter')
        # Select the dataset.
        if self.dataset_name == 'pile':
            data_streamer = load_dataset( "EleutherAI/pile", name="all", split="train", streaming=True )
        elif self.dataset_name == 'red':
            data_streamer = load_dataset("togethercomputer/RedPajama-Data-1T", 'default', split='train', streaming=True)
        else:
            raise RuntimeError( f'{self.dataset_name} is unknown.' )
        
        # Iterate and tokenize the dataset.
        buffer = []
        for sample in data_streamer.shuffle(buffer_size=100):
            text = sample["text"]
            buffer += self.tokenizer(text)["input_ids"]
            buffer += [self.tokenizer.eos_token_id]
            print (text, buffer)
            while len(buffer) > self.sequence_length:
                print (self.tokenizer.decode( torch.tensor(buffer[: self.sequence_length]) ))
                yield torch.tensor(buffer[: self.sequence_length])
                buffer = buffer[self.sequence_length :]

In [3]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
dataset = Dataset( dataset_name = 'pile', tokenizer = tokenizer, sequence_length = 3 )


In [5]:
for b in dataset:
    print (b)
    break

Token indices sequence length is longer than the specified maximum sequence length for this model (6126 > 1024). Running this sequence through the model will result in indexing errors


[21906, 1391, 2, 82, 16, 92, 198, 25609, 198, 198, 3646, 12643, 64, 3519, 11916, 286, 10241, 389, 262, 749, 2219, 19481, 286, 1692, 33593, 13, 9745, 2280, 11, 1844, 35976, 16451, 11, 6825, 35976, 16451, 11, 42465, 35976, 16451, 11, 8556, 35976, 16451, 290, 21957, 2470, 1035, 1648, 19777, 3917, 393, 407, 351, 22160, 32608, 2021, 11916, 2689, 517, 621, 1542, 4, 286, 8668, 33593, 287, 5384, 13, 2312, 21957, 2470, 11916, 389, 15313, 287, 584, 46177, 4693, 685, 31, 79, 505, 13, 405, 1507, 50049, 12, 41, 1942, 544, 2821, 16, 4357, 685, 31, 79, 505, 13, 405, 1507, 50049, 12, 41, 1942, 544, 2821, 17, 4083, 198, 198, 20490, 458, 12643, 341, 318, 16264, 416, 262, 4047, 27376, 3450, 286, 262, 3721, 385, 543, 11525, 82, 2346, 3190, 1626, 262, 22160, 31500, 500, 886, 908, 19172, 290, 31194, 616, 908, 19172, 290, 416, 257, 38977, 9417, 286, 262, 8171, 286, 262, 22160, 23642, 45894, 685, 31, 79, 505, 13, 405, 1507, 50049, 12, 41, 1942, 544, 2821, 16, 4083, 554, 3487, 33593, 11, 262, 14555, 9539, 286,

In [ ]:
# Load dataloader
def load_dataloader():
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation = True, padding = "max_length", max_length = config.sl, return_tensors = "pt")
    dataset = load_dataset("togethercomputer/RedPajama-Data-1T", 'default', split='train', streaming=True)
    dataset = dataset.shuffle(buffer_size = config.bs * 4, seed=42)
    tokenized_dataset = dataset.map( tokenize_function, batched=True )
    dataloader = DataLoader( tokenized_dataset, batch_size = config.bs)
    return dataloader

dataloader = load_dataloader()
pass

In [ ]:
# Get optimized and scheduler
optimizer = torch.optim.AdamW (model.parameters(), lr = config.lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=config.num_warmup, num_training_steps=config.max_steps)  # assuming total steps
pass


In [ ]:

# training loop
step = 0
accumulation_counter = 0
for epoch in range(3):
    print(f'Epoch {epoch + 1}/{3}')
    for batch in dataloader:
        
        # Forward pass.
        outputs = model(
            input_ids = batch["input_ids"].to(device), 
            attention_mask = batch["attention_mask"].to(device),
            labels = batch["input_ids"].to(device)
        ) 
        
        # Backward pass
        loss = outputs.loss / config.accs_per_step
        loss.backward()

        break
    break


In [ ]:
compressed_grads = {}
compressed_sizes = {}
for name, param in model.named_parameters():
    if param.grad is not None:
        element = param.grad.clone().detach().cpu()
        norm, sign_xi_array = utils.compressor.compress(  element  )
        compressed_grads[name] = bt.tensor( tensor = sign_xi_array )
        compressed_sizes[name] = bt.tensor( tensor = norm )

In [ ]:
 grads = {}
for name, compressed_grad in compressed_grads.items():
    compressed_size = compressed_sizes[name]
    grads[name] = compressor.decompress( sign_xi_array = compressed_grad.tensor(), norm = compressed_size.tensor() )
    

In [ ]:
type( sign_xi_array )

In [ ]:

import torch
import torch.quantization

# Define a float tensor
x = torch.rand(3, 3)

# Quantize the tensor to 8 bits
q_x = torch.quantization.quantize_per_tensor(x, scale=1.0, zero_point=0, dtype=torch.quint8)

# You can also dequantize it back to float
dq_x = q_x.dequantize()
